**1. Визначити сутність і виконати аналіз функцій парного блокуючого обміну.**

Парний блокуючий обмін - це базова модель взаємодії двох процесів у MPI, коли один процес надсилає повідомлення, а інший його приймає, і обидві операції є блокуючими, тобто процес зупиняється, доки виклик не завершиться.

У парному обміні беруть участь два процеси: відправник і одержувач.

Відправник викликає функцію MPI_Send, передаючи адресу буфера, тип даних, кількість елементів, тег повідомлення та ранг процесу-одержувача.

Одержувач викликає MPI_Recv з аналогічними параметрами, вказуючи, від кого він чекає повідомлення та який тип і тег має відповідати.

Блокуюча відправка означає, що процес-відправник не продовжить виконання, поки MPI не буде повністю впевнена, що параметри буфера можна безпечно змінювати - або тому, що дані вже гарантовано передані, або тому, що вони скопійовані у внутрішній буфер MPI.

Важливою частиною парного обміну є узгодженість параметрів: обидві сторони мають співпасти за типом даних, розміром та тегом. Це забезпечує коректність і запобігає неоднозначності.

Блокуючі виклики забезпечують ряд важливих властивостей.
 * Кожен MPI_Recv отримає саме те повідомлення, якого він очікує.
 * Вони забезпечують локальну синхронізацію між процесами. Хоча MPI_Send не зобов’язаний чекати фактичного прийому, у більшості реалізацій прості повідомлення передаються одразу, і відправник блокується тільки до моменту копіювання в буфер MPI.
 * Парний блокуючий обмін може бути джерелом взаємних блокувань (deadlock), якщо обидва процеси викликають MPI_Send, не маючи активних прийомів. Наприклад, двостороння відправка без попереднього MPI_Recv може заблокувати програму, якщо реалізація MPI не має достатньо буфера для обох повідомлень.
 * Також, важливо розуміти його семантику: завершення відправки не означає, що повідомлення доставлене одержувачу. Завершення прийому означає, що дані гарантовано знаходяться у буфері одержувача.

In [1]:
!pip install mpi4py


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


**Тестова програма для перевірки роботи бібліотеки в середовищі Jupiter Notebook**

In [2]:
%%writefile test_mpi.py
from mpi4py import MPI
comm = MPI.COMM_WORLD
print(f"Rank: {comm.Get_rank()} | Size: {comm.Get_size()}")

Overwriting test_mpi.py


In [3]:
!mpiexec -n 4 python test_mpi.py

Rank: 1 | Size: 4
Rank: 0 | Size: 4


Rank: 2 | Size: 4
Rank: 3 | Size: 4




**2. Побудувати програму-шаблон для парного блокуючого обміну**

In [4]:
%%writefile task2.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code"
    numProcesses = comm.Get_size()  #total number of processes running"
    myHostName = MPI.Get_processor_name()  #machine name running the code"

    # num of processes must be even
    if numProcesses > 1 and not odd(numProcesses):
        sendValue = id

         #odd processes receive from their paired 'neighbor', then send
        if odd(id):
            comm.send(sendValue, dest=id - 1)
            receivedValue = comm.recv(source=id - 1)

        #even processes receive from their paired 'neighbor', then send
        else:
            receivedValue = comm.recv(source=id + 1)
            comm.send(sendValue, dest=id + 1)

        print(
            "Process {} of {} on {} computed {} and received {}".format(
                id, numProcesses, myHostName, sendValue, receivedValue
            )
        )

    else:
        if id == 0:
            print("Please run this program with a positive even number of processes.")

if __name__ == "__main__":
    main()


Overwriting task2.py


In [5]:
!mpiexec -n 6 python task2.py

Process 2 of 6 on DESKTOP-L6FRMTM computed 2 and received 3
Process 4 of 6 on DESKTOP-L6FRMTM computed 4 and received 5

Process 3 of 6 on DESKTOP-L6FRMTM computed 3 and received 2

Process 5 of 6 on DESKTOP-L6FRMTM computed 5 and received 4


Process 0 of 6 on DESKTOP-L6FRMTM computed 0 and received 1
Process 1 of 6 on DESKTOP-L6FRMTM computed 1 and received 0


Як можемо бачити з цього прикладу, дані попарно відправляються між процесами один одному.

Процес 1, відправив процесу 0 значення свого рангу "1", а процес 0 в свою чергу, відправив процесу 1 аналогічно - "0"

**Атрибути функцій обміну (Send / Recv):**

У блокуючому парному обміні беруть участь такі параметри:

* buf - область памʼяті з даними для передачі або прийому.
* count - кількість елементів у буфері.
* datatype - MPI-тип одного елемента (MPI_INT, MPI_DOUBLE, масиви NumPy і т.д.).
* dest / source - ранг процесу, якому надсилаємо або від якого очікуємо повідомлення.
* tag - ціле число для логічної ідентифікації виду повідомлення.
* comm - комунікатор (наприклад, MPI_COMM_WORLD).
* status (тільки для Recv) - структура, у якій після завершення зберігається фактичний відправник, тег та код завершення.

**Аналіз типів даних:**
1. MPI не робить автоматичного перетворення типів: тип відправника і одержувача має збігатися.
2. count і datatype однозначно визначають обсяг памʼяті, що MPI повинна передати.
3. Якщо count = 0, операція дозволена й завершується миттєво.
4. Розмір приймального буфера повинен бути не меншим, ніж фактичний розмір отриманих даних - інакше помилка.
5. У mpi4py:
  * send(obj) - обʼєкт серіалізується, тип визначається автоматично.
  * Send([...], MPI.INT) - передаються сирі дані з чітким типом і розміром.
6. Типи даних визначають спосіб розбору байтів, тому невідповідність типів призводить до некоректного читання або помилки.

**Семантичний аналіз парного блокуючого обміну**

1. Операція MPI_Send блокується, поки MPI не гарантує, що буфер можна змінювати (дані або передані, або скопійовані в буфер MPI).
2. MPI_Recv блокується, поки дані не будуть повністю отримані і записані в приймальний буфер.
3. Відправник та одержувач повинні бути логічно узгоджені: однаковий tag, однаковий datatype, однаковий count, коректно вказані source/dest
4. Порушення симетрії веде до дедлоку (наприклад, обидва процеси одночасно викликають Send і жоден не виконує Recv).
5. Завершення Send не означає, що одержувач вже отримав повідомлення - лише те, що відправник може використовувати буфер.
6. Завершення Recv означає, що дані гарантовано у приймальному буфері.
7. Парний блокуючий обмін створює локальну синхронізацію між процесами: один процес не може рухатись далі, доки пара не виконає узгоджену операцію.

**Змінимо в програмі порядок виконання функцій для парних рангів, щоб продемострувати дедлок**

In [6]:
%%writefile task2_deadlock.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code"
    numProcesses = comm.Get_size()  #total number of processes running"
    myHostName = MPI.Get_processor_name()  #machine name running the code"

    # num of processes must be even
    if numProcesses > 1 and not odd(numProcesses):
        sendValue = id

         #odd processes receive from their paired 'neighbor', then send
        if odd(id):
            receivedValue = comm.recv(source=id - 1)
            comm.send(sendValue, dest=id - 1)

        #even processes receive from their paired 'neighbor', then send
        else:
            receivedValue = comm.recv(source=id + 1)
            comm.send(sendValue, dest=id + 1)

        print(
            "Process {} of {} on {} computed {} and received {}".format(
                id, numProcesses, myHostName, sendValue, receivedValue
            )
        )

    else:
        if id == 0:
            print("Please run this program with a positive even number of processes.")

if __name__ == "__main__":
    main()


Overwriting task2_deadlock.py


In [7]:
!mpiexec -n 6 python task2_deadlock.py

^C


Як бачимо процеси зависли - дедлок

**3. Виконати аналіз інших комунікаційних режимів. Показати зміни в шаблоні для їх реалізації.**

**У MPI існує чотири режими відправки повідомлень. Головна відмінність між ними — це умова завершення функції відправки (коли функція повертає управління програмі)**

* Синхронний (Synchronous), ssend - Рукостискання. Функція завершується тільки тоді, коли процес-одержувач розпочав прийом повідомлення (викликав recv). Це гарантує, що обидва процеси досягли точки обміну. Найбезпечніший, але може бути повільнішим через очікування
* Буферизований (Buffered), bsend - Через буфер користувача. Повідомлення копіюється у спеціально виділений  буфер, і функція повертається миттєво. Не треба залежати від того, чи готовий одержувач. Ризик: якщо буфер переповниться, виникне помилка.
* По готовності (Ready), rsend - Оптимістичний. Можна викликати лише якщо ви впевнені на 100%, що одержувач вже викликав recv. Якщо ні — поведінка не визначена (помилка або краш). Використовується вкрай рідко для оптимізації.

In [22]:
%%writefile task3.py
from mpi4py import MPI
import numpy as np

# Функція перевірки на непарність
def odd(number):
    return number % 2 != 0

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    name = MPI.Get_processor_name()

    # Перевірка, що кількість процесів парна
    if size < 2 or odd(size):
        if rank == 0:
            print("Please run this program with a positive even number of processes.")
        exit(0)

    # Підготовка даних (використовуємо numpy, як у вашому прикладі на фото)
    # rank - це число, яке ми надсилаємо
    send_data = np.array([rank], dtype='i')
    recv_data = np.array([0], dtype='i')

    # Визначаємо сусіда
    if odd(rank):
        neighbor = rank - 1
    else:
        neighbor = rank + 1

    # === ЛОГІКА ОБМІНУ ===

    # Для простоти реалізуємо схему:
    # Непарні: ВІДПРАВЛЯЮТЬ всіма способами -> потім ПРИЙМАЮТЬ всіма способами
    # Парні:   ПРИЙМАЮТЬ всіма способами   -> потім ВІДПРАВЛЯЮТЬ всіма способами
    # Це запобігає дедлокам і дозволяє Rsend працювати коректно (бо Recv вже викликаний).

    if odd(rank):
        # --- 1. ВІДПРАВКА (Send) ---
        # Standard
        comm.Send([send_data, MPI.INT], dest=neighbor, tag=0)

        # Buffered
        buf = bytearray(MPI.BSEND_OVERHEAD + send_data.nbytes)
        MPI.Attach_buffer(buf)
        comm.Bsend([send_data, MPI.INT], dest=neighbor, tag=1)
        MPI.Detach_buffer()

        # Synchronous
        comm.Ssend([send_data, MPI.INT], dest=neighbor, tag=2)

        # Ready (працює, бо парний процес вже чекає на Recv)
        comm.Rsend([send_data, MPI.INT], dest=neighbor, tag=3)

        print(f"Process {rank} finished ALL SENDS to {neighbor}")

        # --- 2. ПРИЙОМ (Receive) ---
        comm.Recv([recv_data, MPI.INT], source=neighbor, tag=0) # Standard
        comm.Recv([recv_data, MPI.INT], source=neighbor, tag=1) # Buffered
        comm.Recv([recv_data, MPI.INT], source=neighbor, tag=2) # Sync
        comm.Recv([recv_data, MPI.INT], source=neighbor, tag=3) # Ready

        print(f"Process {rank} finished ALL RECVS from {neighbor}. Last value: {recv_data[0]}")

    else:
        # --- 1. ПРИЙОМ (Receive) ---
        # Парні процеси спочатку слухають, що робить їх готовими до Rsend від сусідів
        comm.Recv([recv_data, MPI.INT], source=neighbor, tag=0) # Standard
        comm.Recv([recv_data, MPI.INT], source=neighbor, tag=1) # Buffered
        comm.Recv([recv_data, MPI.INT], source=neighbor, tag=2) # Sync
        comm.Recv([recv_data, MPI.INT], source=neighbor, tag=3) # Ready

        print(f"Process {rank} finished ALL RECVS from {neighbor}. Last value: {recv_data[0]}")

        # --- 2. ВІДПРАВКА (Send) ---
        # Standard
        comm.Send([send_data, MPI.INT], dest=neighbor, tag=0)

        # Buffered
        buf = bytearray(MPI.BSEND_OVERHEAD + send_data.nbytes)
        MPI.Attach_buffer(buf)
        comm.Bsend([send_data, MPI.INT], dest=neighbor, tag=1)
        MPI.Detach_buffer()

        # Synchronous
        comm.Ssend([send_data, MPI.INT], dest=neighbor, tag=2)

        # Ready
        comm.Rsend([send_data, MPI.INT], dest=neighbor, tag=3)

        print(f"Process {rank} finished ALL SENDS to {neighbor}")

if __name__ == "__main__":
    main()

Overwriting task3.py


In [27]:
!mpiexec -n 6 python task3.py

Process 1 finished ALL SENDS to 0
Process 0 finished ALL RECVS from 1. Last value: 1


Process 0 finished ALL SENDS to 1
Process 1 finished ALL RECVS from 0. Last value: 0


Process 5 finished ALL SENDS to 4
Process 4 finished ALL RECVS from 5. Last value: 5


Process 4 finished ALL SENDS to 5
Process 5 finished ALL RECVS from 4. Last value: 4


Process 3 finished ALL SENDS to 2
Process 2 finished ALL RECVS from 3. Last value: 3


Process 2 finished ALL SENDS to 3
Process 3 finished ALL RECVS from 2. Last value: 2




**4. Сутність та аналіз функцій парного неблокуючого обміну**

Сутність неблокуючого обміну Неблокуючий обмін - це техніка, яка дозволяє процесу ініціювати операцію передачі або прийому даних і негайно продовжити виконання програми, не чекаючи фактичного завершення цієї операції. На відміну від блокуючого режиму, де процес "зависає" до моменту, поки дані не будуть безпечно скопійовані, неблокуючий режим розділяє комунікацію на дві фази:

1. Ініціалізація: Виклик функцій початку обміну (MPI_Isend, MPI_Irecv). Ці функції лише повідомляють системі про намір передати/прийняти дані та миттєво повертають керування програмі, повертаючи спеціальний об'єкт - запит (Request).
2. Завершення: Перевірка стану операції за допомогою отриманого об'єкта запиту. Для цього використовуються функції очікування MPI_Wait (блокує процес до завершення) або перевірки MPI_Test (повертає статус, не блокуючи процес).

**Аналіз функцій**

MPI_Isend / comm.isend: Ініціює відправку. Буфер з даними не можна змінювати після цього виклику до моменту успішного завершення операції (виклику Wait), оскільки система може все ще читати з нього дані.

MPI_Irecv / comm.irecv: Ініціює прийом. Спроба прочитати дані з буфера до завершення операції призведе до отримання сміття, оскільки дані могли ще не надійти.

MPI_Wait / req.wait: Гарантує завершення операції. Повернення з цієї функції означає, що буфер відправника можна перезаписувати, або що в буфері одержувача вже лежать коректні дані.

In [28]:
%%writefile task4.py
from mpi4py import MPI
import time

def odd(number):
    return number % 2 != 0

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    name = MPI.Get_processor_name()

    # Перевірка на парність кількості процесів
    if size < 2 or odd(size):
        if rank == 0:
            print("Error: Please run with an even number of processes.")
        return

    # Визначаємо сусіда
    neighbor = rank - 1 if odd(rank) else rank + 1

    send_val = rank
    print(f"[P{rank}] Initialized. Prepare to exchange with P{neighbor}")

    # === НЕБЛОКУЮЧИЙ ОБМІН ===

    # 1. Ініціалізація ПРИЙОМУ (Irecv)
    # Функція повертає об'єкт Request, але не чекає даних
    print(f"[P{rank}] Starting non-blocking Receive...")
    req_recv = comm.irecv(source=neighbor, tag=11)

    # 2. Ініціалізація ВІДПРАВКИ (Isend)
    # Функція повертає об'єкт Request і миттєво йде далі
    print(f"[P{rank}] Starting non-blocking Send ({send_val})...")
    req_send = comm.isend(send_val, dest=neighbor, tag=11)

    # 3. КОРИСНА РОБОТА (Overlapping)
    # У цей час, поки дані "летять" по мережі, процесор не простоює.
    # Ми можемо виконувати обчислення.
    print(f"[P{rank}] Doing some calculations while waiting...")
    time.sleep(1) # Імітація обчислень

    # 4. ОЧІКУВАННЯ ЗАВЕРШЕННЯ (Wait)
    # Тепер нам потрібні результати, тому ми явно чекаємо завершення.

    # Чекаємо, поки відправка точно завершиться (буфер стане вільним)
    req_send.wait()
    print(f"[P{rank}] Send completed.")

    # Чекаємо, поки дані точно прийдуть
    received_val = req_recv.wait()
    print(f"[P{rank}] Receive completed. Got data: {received_val}")

    # Фінальний вивід
    print(f"SUCCESS: Process {rank} on {name} | Sent: {send_val} | Received: {received_val}")

if __name__ == "__main__":
    main()

Writing task4.py


In [29]:
!mpiexec -n 6 python task4.py

[P3] Initialized. Prepare to exchange with P2
[P3] Starting non-blocking Receive...
[P5] Initialized. Prepare to exchange with P4
[P5] Starting non-blocking Receive...
[P2] Initialized. Prepare to exchange with P3
[P2] Starting non-blocking Receive...
[P4] Initialized. Prepare to exchange with P5
[P4] Starting non-blocking Receive...
[P3] Starting non-blocking Send (3)...
[P5] Starting non-blocking Send (5)...
[P2] Starting non-blocking Send (2)...
[P4] Starting non-blocking Send (4)...




[P0] Initialized. Prepare to exchange with P1
[P1] Initialized. Prepare to exchange with P0
[P0] Starting non-blocking Receive...


[P1] Starting non-blocking Receive...
[P0] Starting non-blocking Send (0)...


[P4] Doing some calculations while waiting...
[P5] Doing some calculations while waiting...
[P2] Doing some calculations while waiting...
[P3] Doing some calculations while waiting...
[P1] Starting non-blocking Send (1)...
[P0] Doing some calculations while waiting...






[P1] Doing some ca